In [2]:
import re
def regex_pos(post_text):
    x1 = re.findall('[A-Z]{1,4}\s\d{1,3}[cCpP]\s\d{1,2}/\d{1,4}', post_text)
    x2 = re.findall('\$?[A-Z]{1,4}\s\d{1,2}\/\d{1,4}\s\$?\d{1,3}\s\w{1,4}', post_text)
    x = x1.extend(x2)
    if x: return x
    return []

In [3]:
import json
import os
from tqdm import tqdm

In [15]:
with open('Example Data - Testing/1.10.2019.15.30.32.json', 'r') as f:
    posts = json.load(f)
    numposts = 0
    textposts = 0
    positions = 0
    for post in posts:
        numposts += 1
        if 'selftext' in post:
            textposts += 1
            tmp = regex_pos(post['selftext'])
            if tmp:
                positions += len(tmp)
                print(tmp)
    print('Total posts in JSON: {}'.format(numposts))
    print('Posts with text: {}'.format(textposts))
    print('Total positions found: {}'.format(positions))
    print('Success rate (out of posts with with text): {}%'.format(round(positions/textposts * 100,2)))

Total posts in JSON: 34
Posts with text: 34
Total positions found: 0
Success rate (out of posts with with text): 0.0%


In [16]:
with open('Example Data - Testing/1.10.2019.21.5.39.json', 'r') as f:
    posts = json.load(f)
    numposts = 0
    textposts = 0
    positions = 0
    for post in posts:
        numposts += 1
        if 'selftext' in post:
            textposts += 1
            tmp = regex_pos(post['selftext'])
            if tmp:
                positions += len(tmp)
                print(tmp)
    print('Total posts in JSON: {}'.format(numposts))
    print('Posts with text: {}'.format(textposts))
    print('Total positions found: {}'.format(positions))
    print('Success rate (out of posts with with text): {}%'.format(round(positions/textposts * 100,2)))

Total posts in JSON: 33
Posts with text: 32
Total positions found: 0
Success rate (out of posts with with text): 0.0%


In [6]:
directory = 'post_data'
numposts = 0
textposts = 0
positions = 0
regexed = []
for file in os.scandir(directory):
    with open(file.path, 'r') as f:
        posts = json.load(f)
        for post in posts:
            numposts += 1
            if 'selftext' in post:
                textposts += 1
                tmp = regex_pos(post['selftext'])
                if tmp:
                    positions += len(tmp)
                    regexed.append(tmp)
print('Total posts in JSON: {}'.format(numposts))
print('Posts with text: {}'.format(textposts))
print('Total positions found: {}'.format(positions))
print('Success rate (out of posts with with text): {}%'.format(round(positions/textposts * 100,2)))

Total posts in JSON: 109912
Posts with text: 107919
Total positions found: 0
Success rate (out of posts with with text): 0.0%


In [5]:
## lets make the attempt with nltk and see what we get
from nltk.corpus import stopwords
from nltk import FreqDist
import collections
import string

def clean_post(post_text):
    translator = str.maketrans('', '', string.punctuation) # for removing punctuation
    post_text = post_text.translate(translator)
    token_text = [t for t in post_text.replace('\n',' ').split()]
    
    sr = stopwords.words('english')
    cleanTokens = [i for i in token_text if i not in sr]
    
    return cleanTokens

In [6]:
directory = 'post_data'
numposts = 0
textposts = 0
freq_list = []
for file in os.scandir(directory):
    with open(file.path, 'r') as f:
        posts = json.load(f)
        for post in posts:
            numposts += 1
            if 'selftext' in post:
                textposts += 1
                freq = FreqDist(clean_post(post['selftext']))
                freq_list.append(freq.most_common(100))
freq_list

[[('likely', 11),
  ('Most', 10),
  ('PAPA', 2),
  ('JOHN', 2),
  ('company', 2),
  ('bad', 2),
  ('Please', 1),
  ('add', 1),
  ('I', 1),
  ('forgot', 1),
  ('nudes', 1),
  ('leaked', 1),
  ('multibillion', 1),
  ('dollar', 1),
  ('stolen', 1),
  ('Robinhood', 1),
  ('commit', 1),
  ('murder', 1),
  ('Boeing', 1),
  ('comeback', 1),
  ('ruined', 1),
  ('Russia', 1),
  ('Facebook', 1),
  ('wreck', 1),
  ('portfolio', 1),
  ('toilet', 1),
  ('tweet', 1),
  ('President', 1),
  ('Donald', 1),
  ('J', 1),
  ('Trump', 1),
  ('press', 1),
  ('sponsored', 1),
  ('state', 1),
  ('California', 1),
  ('PGampE', 1),
  ('continue', 1),
  ('shitty', 1),
  ('Blue', 1),
  ('Apron', 1),
  ('hard', 1),
  ('correction', 1),
  ('Netflix', 1),
  ('fired', 1),
  ('Masayoshi', 1),
  ('Son', 1),
  ('Another', 1),
  ('still', 1),
  ('piece', 1),
  ('shit', 1),
  ('GameStop', 1),
  ('movie', 1),
  ('made', 1),
  ('innuendos', 1),
  ('capitalism', 1),
  ('need', 1),
  ('socializm', 1),
  ('WeWork', 1)],
 [],
 [

In [7]:
post

{'all_awardings': [],
 'allow_live_comments': False,
 'author': 'bleubeard',
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_text': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_5yc89',
 'author_patreon_flair': False,
 'author_premium': False,
 'awarders': [],
 'can_mod_post': False,
 'contest_mode': False,
 'created_utc': 1599641354,
 'domain': 'bloomberg.com',
 'full_link': 'https://www.reddit.com/r/wallstreetbets/comments/ipcbek/astrazeneca_vaccine_on_hold_load_your_puts_on_azn/',
 'gildings': {},
 'id': 'ipcbek',
 'is_crosspostable': False,
 'is_meta': False,
 'is_original_content': False,
 'is_reddit_media_domain': False,
 'is_robot_indexable': False,
 'is_self': False,
 'is_video': False,
 'link_flair_background_color': '#800080',
 'link_flair_css_class': 'question',
 'link_flair_richtext': [{'e': 'text', 't': 'Discussion'}],
 'link_flair_template_id': '96f6c79e-b853-11e5-a4cb-0ebdf030e05d',
 'link_flair_text': 'Discussion',
 'link_flai

In [21]:
import json
import pandas as pd
i = 0

for file in tqdm(os.scandir('post_data')):
    with open(file.path, 'r') as f:
        if i == 0:
            data = json.load(f)
        else:
            data = data + json.load(f)
    i += 1


56517it [01:01, 915.50it/s] 


In [24]:
pd.DataFrame(data)['selftext']

0         Please add any that I forgot:\n\nMost likely t...
1                                                          
2         Please add any that I forgot:\n\nMost likely t...
3                                                 [removed]
4                                                          
                                ...                        
109907                                                     
109908                                            [removed]
109909    Excuse the “flair”, idk what to use. I don’t r...
109910                                            [removed]
109911                                                     
Name: selftext, Length: 109912, dtype: object